In [1]:
import sys
from time import sleep

import pygame

from settings import Settings
from game_stats import GameStats
from scoreboard import Scoreboard
from button import Button
from ship import Ship
from bullet import Bullet
from alien import Alien


pygame 2.1.2 (SDL 2.0.18, Python 3.8.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
class AlienInvasion:
    """Overall class to manage game assets and behavior."""

    def __init__(self):
        """Initialize the game, and create game resources."""
        pygame.init()
        self.settings = Settings()

        self.screen = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)#full screen
        self.settings.screen_width  = self.screen.get_rect().width
        self.settings.screen_height  = self.screen.get_rect().height
        pygame.display.set_caption("Alien Invasion")

        # Create an instance to store game statistics,
        #   and create a scoreboard.
        self.stats = GameStats(self)#The "GameStats" and "Scoreboard" classes are custom classes that are not part of Pygame.
        self.sb = Scoreboard(self)

        self.ship = Ship(self)#custom classes 
        self.bullets = pygame.sprite.Group()#The code also creates an empty group for storing the "Bullet" objects and an empty group for storing the "Alien" objects.
        self.aliens = pygame.sprite.Group()

        self._create_fleet()#custom method After creating the ship and groups, the method calls "_create_fleet()" to create the fleet of aliens.

        # Make the Play button.
        self.play_button = Button(self, "Play")#custom class see update screen

    def run_game(self):
        """Start the main loop for the game."""
        while True:
            self._check_events()#In each iteration of the loop, the method calls "_check_events()" to detect and handle any events, such as key presses or mouse clicks.

            if self.stats.game_active:#line no. 10 game_statIf the "game_active" attribute of the "GameStats" instance is True
                self.ship.update()
                self._update_bullets()#method
                self._update_aliens()

            self._update_screen()# update the display on the screen

    def _check_events(self):
        """Respond to keypresses and mouse events."""#
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                SystemExit()
            elif event.type == pygame.KEYDOWN:
                self._check_keydown_events(event)
            elif event.type == pygame.KEYUP:
                self._check_keyup_events(event)
            elif event.type == pygame.MOUSEBUTTONDOWN:#mouse  button sy play button ko click kry gy
                mouse_pos = pygame.mouse.get_pos()
                self._check_play_button(mouse_pos)

    def _check_play_button(self, mouse_pos):
        """Start a new game when the player clicks Play."""
        """This code checks if the play button was clicked. 
        If it was, and the game is not already active, 
        it resets the game settings and statistics, 
        removes any remaining aliens and bullets, 
        creates a new fleet of aliens, centers the ship,
        and hides the mouse cursor. The purpose of this 
        code is to start a new game when the player clicks the "Play" button."""
        button_clicked = self.play_button.rect.collidepoint(mouse_pos)
        if button_clicked and not self.stats.game_active:
            # Reset the game settings.
            self.settings.initialize_dynamic_settings()

            # Reset the game statistics.
            self.stats.reset_stats()
            self.stats.game_active = True
            self.sb.prep_score()
            self.sb.prep_level()
            self.sb.prep_ships()

            # Get rid of any remaining aliens and bullets.
            self.aliens.empty()
            self.bullets.empty()
            
            # Create a new fleet and center the ship.
            self._create_fleet()
            self.ship.center_ship()

            # Hide the mouse cursor.
            pygame.mouse.set_visible(False)

    def _check_keydown_events(self, event):
        """Respond to keypresses."""
        if event.key == pygame.K_RIGHT:#When the right arrow key is pressed, the moving_right attribute of the ship object is set to True
            self.ship.moving_right = True
        elif event.key == pygame.K_LEFT:#When the left arrow key is pressed, the moving_left attribute of the ship object is set to True.
            self.ship.moving_left = True
        elif event.key == pygame.K_q:#When the 'q' key is pressed, the function calls SystemExit() which terminates the game.
            SystemExit()
        elif event.key == pygame.K_SPACE:#fire bullets using spacebar
            self._fire_bullet()

    def _check_keyup_events(self, event):#This code checks for key release events, 
        """Respond to key releases."""
        if event.key == pygame.K_RIGHT:#If the right arrow key is released, the moving_right attribute of the ship object is set to False
            self.ship.moving_right = False
        elif event.key == pygame.K_LEFT:# left arrow key is released, the moving_left attribute of the ship object is set to False
            self.ship.moving_left = False

    def _fire_bullet(self):
        """Create a new bullet and add it to the bullets group."""
        if len(self.bullets) < self.settings.bullets_allowed:#in seting.py allowed bullets are given=4
            new_bullet = Bullet(self)#create new bullets
            self.bullets.add(new_bullet)#"""self.bullet ka upar group bnaya""" upar self.bullets attribute me add kr dy gy

    def _update_bullets(self):#The _update_bullets method updates the position of the bullets and removes any bullets that have disappeared (i.e., those that have gone off the top of the screen).
        """Update position of bullets and get rid of old bullets."""
        # Update bullet positions.
        self.bullets.update()#It first updates the position of all the bullets in the self.bullets group using the update method.

        
        for bullet in self.bullets.copy():# Get rid of bullets that have disappeared.
            if bullet.rect.bottom <= 0:#Then, it iterates over a copy of the self.bullets group to remove any bullets that have a rect.bottom value less than or equal to 0.
                 self.bullets.remove(bullet)

        self._check_bullet_alien_collisions()

    def _check_bullet_alien_collisions(self):
        """The method _check_bullet_alien_collisions checks
        for collisions between the bullets and aliens. If a 
        collision is detected, the colliding bullet and alien are removed.
        The score is updated based on the number of aliens that have been 
        destroyed and the value of each alien as specified in self.settings.alien_points.
        The score is prepared for display on the screen and checked against the high score.
        If all the aliens have been destroyed, the existing bullets are destroyed and a new fleet 
        of aliens is created. The game speed is also increased and the level is incremented."""
        """Respond to bullet-alien collisions."""
        # Remove any bullets and aliens that have collided.
        collisions = pygame.sprite.groupcollide(
                self.bullets, self.aliens, True, True)

        if collisions:
            for aliens in collisions.values():
                self.stats.score += self.settings.alien_points * len(aliens)
            self.sb.prep_score()
            self.sb.check_high_score()

        if not self.aliens:
            # Destroy existing bullets and create new fleet.
            self.bullets.empty()
            self._create_fleet()
            self.settings.increase_speed()

            # Increase level.
            self.stats.level += 1
            self.sb.prep_level()

    def _update_aliens(self):
        """
        This function updates the position of the aliens in the game.
        It first checks if the entire fleet of aliens is at the edge of 
        the screen and updates their positions accordingly.
        Then, it checks if any of the aliens have collided with the ship.
        If a collision is detected, the function calls the _ship_hit method.
        Finally, it checks if any of the aliens have reached the bottom of the screen.
        If they have, it calls the _check_aliens_bottom method.
        """
        self._check_fleet_edges()
        self.aliens.update()

        # Look for alien-ship collisions.
        if pygame.sprite.spritecollideany(self.ship, self.aliens):
            self._ship_hit()

        # Look for aliens hitting the bottom of the screen.
        self._check_aliens_bottom()

    def _check_aliens_bottom(self):
        """The function _check_aliens_bottom checks 
        if any of the aliens in the fleet have reached 
        the bottom of the screen. If an alien reaches the bottom,
        the function treats this as if the ship has been hit and calls the _ship_hit function."""
        screen_rect = self.screen.get_rect()
        for alien in self.aliens.sprites():
            if alien.rect.bottom >= screen_rect.bottom:
                # Treat this the same as if the ship got hit.
                self._ship_hit()
                break

    def _ship_hit(self):
        """This code block implements what happens when the ship collides with an alien.
        If there are still ships left (as stored in self.stats.ships_left),
        the number of ships left is decremented, the scoreboard is updated, 
        the current aliens and bullets on the screen are removed, a new fleet of aliens is created,
        and the ship is centered. The game is then paused for half a second. 
        If there are no ships left, the game is set to inactive, and the mouse pointer is made visible."""
        if self.stats.ships_left > 0:
            # Decrement ships_left, and update scoreboard.
            self.stats.ships_left -= 1#when collide one ship will decrease
            self.sb.prep_ships()
            
            # Get rid of any remaining aliens and bullets.
            self.aliens.empty()
            self.bullets.empty()
            
            # Create a new fleet and center the ship.
            self._create_fleet()
            self.ship.center_ship()
            
            # Pause.
            sleep(0.5)
        else:
            self.stats.game_active = False
            pygame.mouse.set_visible(True)

    def _create_fleet(self):
        """The code creates a fleet of aliens on the screen.

It first creates an instance of the Alien class.
Then it calculates the width and height of an alien and the available space on the X and Y axis.
Using the available space on the X axis, it calculates the number of aliens that can fit in a row.
Using the available space on the Y axis, it calculates the number of rows of aliens that can fit on the screen.
The code does not actually create the aliens, it only calculates the number of aliens that can fit in a row and
the number of rows of aliens that can fit on the screen."""
        # Create an alien and find the number of aliens in a row.
        # Spacing between each alien is equal to one alien width.
        alien = Alien(self)
        alien_width, alien_height = alien.rect.size
        available_space_x = self.settings.screen_width - (2 * alien_width)
        number_aliens_x = available_space_x // (2 * alien_width)
        
        # Determine the number of rows of aliens that fit on the screen.
        ship_height = self.ship.rect.height
        available_space_y = (self.settings.screen_height -
                                (3 * alien_height) - ship_height)
        number_rows = available_space_y // (2 * alien_height)
        
        # Create the full fleet of aliens.
        for row_number in range(number_rows):
            for alien_number in range(number_aliens_x):
                self._create_alien(alien_number, row_number)

    def _create_alien(self, alien_number, row_number):
        """this is the method to create an individual alien in the fleet. 
        The alien_number argument is used to place the alien in the correct 
        horizontal position within the row, while the row_number argument is 
        used to place the alien in the correct vertical position on the screen. 
        The method first creates an instance of the Alien class and calculates the 
        width and height of the alien. Then it sets the x-coordinate of the alien based 
        on the alien_number and the width of the alien. Finally, it sets the y-coordinate of 
        the alien based on the row_number and the height of the alien, and adds the alien to the 
        self.aliens sprite group.."""
        alien = Alien(self)
        alien_width, alien_height = alien.rect.size
        alien.x = alien_width + 2 * alien_width * alien_number
        alien.rect.x = alien.x
        alien.rect.y = alien.rect.height + 2 * alien.rect.height * row_number
        self.aliens.add(alien)

    def _check_fleet_edges(self):
        """This code is checking if any of the aliens in a fleet
        have reached the edge of the screen. It does this by looping 
        through all aliens in the fleet and checking each one's position 
        using the check_edges method. If any of the aliens have reached
        the edge, the code calls the _change_fleet_direction method to change
        the direction of the entire fleet."""
        for alien in self.aliens.sprites():
            if alien.check_edges():
                self._change_fleet_direction()
                break
            
    def _change_fleet_direction(self):
        """This is a method _change_fleet_direction in a class 
        related to an alien invasion game. It is used to change the 
        direction of the fleet of aliens by dropping the entire fleet 
        down and reversing the direction in which the fleet is moving. 
        The rect.y attribute of each alien is updated to move the aliens 
        downward by the value specified in self.settings.fleet_drop_speed,
        and the self.settings.fleet_direction attribute is multiplied by -1 
        to reverse the direction of the fleet's movement"""
        for alien in self.aliens.sprites():
            alien.rect.y += self.settings.fleet_drop_speed
        self.settings.fleet_direction *= -1

    def _update_screen(self):
        """This method _update_screen() updates the display 
        of the game screen by first filling the screen with the background color
        defined in the game's settings. It then draws the ship and the bullets on the screen,
        followed by the aliens. The score is also displayed on the screen through the 
        show_score() method of the scoreboard object sb. If the game is inactive 
        (indicated by stats.game_active being False), a play button is also drawn on the screen. 
        Finally, the display is flipped to show the updated screen."""
        self.screen.fill(self.settings.bg_color)
        self.ship.blitme()
        for bullet in self.bullets.sprites():
            bullet.draw_bullet()
        self.aliens.draw(self.screen)

        # Draw the score information.
        self.sb.show_score()

        # Draw the play button if the game is inactive.
        if not self.stats.game_active:
            self.play_button.draw_button()

        pygame.display.flip()#jo screen bnai usy visible krny  lye


if __name__ == '__main__':
    # Make a game instance, and run the game.
    ai = AlienInvasion()
    ai.run_game()
